PAPER : https://drive.google.com/open?id=1fnF-QsiQeKaxF-HbvFiGtzHF_Bf3CzJu

Video : https://drive.google.com/open?id=1f_gbKW6FUUHKkUxciJ_lQx29mCq_fSBy

Reposiroty : https://github.com/NVlabs/stylegan2

In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd "/content/drive/My Drive/stylegan2"

Mounted at /content/drive
/content/drive/My Drive/stylegan2


In [2]:
!git clone https://github.com/dvschultz/stylegan2 # (originally from NVIDIA Labs https://github.com/NVlabs/stylegan2) https://www.nvidia.com/en-us/research/
%cd stylegan2
!mkdir pkl
%cd pkl
!gdown --id 1JLqXE5bGZnlu2BkbLPD5_ZxoO3Nz-AvF #inception model: https://drive.google.com/open?id=1JLqXE5bGZnlu2BkbLPD5_ZxoO3Nz-AvF
%cd ../
!mkdir results
!mkdir results/pretrained
%cd results/pretrained
!gdown --id 1UlDmJVLLnBD9SnLSMXeiZRO6g-OMQCA_
!mv stylegan2-ffhq-config-f.pkl network-snapshot-10000.pkl
%cd ../../
%mkdir datasets

Cloning into 'stylegan2'...
remote: Enumerating objects: 552, done.
remote: Total 552 (delta 0), reused 0 (delta 0), pack-reused 552
Receiving objects: 100% (552/552), 22.47 MiB | 20.03 MiB/s, done.
Resolving deltas: 100% (285/285), done.
/content/drive/My Drive/stylegan2/stylegan2
/content/drive/My Drive/stylegan2/stylegan2/pkl
Downloading...
From: https://drive.google.com/uc?id=1JLqXE5bGZnlu2BkbLPD5_ZxoO3Nz-AvF
To: /content/drive/My Drive/stylegan2/stylegan2/pkl/inception_v3_features.pkl
87.3MB [00:01, 63.8MB/s]
/content/drive/My Drive/stylegan2/stylegan2
/content/drive/My Drive/stylegan2/stylegan2/results/pretrained
Downloading...
From: https://drive.google.com/uc?id=1UlDmJVLLnBD9SnLSMXeiZRO6g-OMQCA_
To: /content/drive/My Drive/stylegan2/stylegan2/results/pretrained/stylegan2-ffhq-config-f.pkl
382MB [00:02, 128MB/s]
/content/drive/My Drive/stylegan2/stylegan2


In [3]:
!ls

datasets		results
dataset_tool.py		run_generator.py
dnnlib			run_metrics.py
Dockerfile		run_projector.py
docs			run_training.py
easy-install.sh		runway_model.py
LICENSE.txt		runway.yml
metrics			StyleGAN2_Google_Colab_Example.ipynb
pkl			StyleGAN2_projection_interpolation.ipynb
pretrained_networks.py	StyleGAN2_Projection.ipynb
projector.py		test_nvcc.cu
README.md		training
requirements.txt	USAGE.md


In [4]:
!pwd

/content/drive/My Drive/stylegan2/stylegan2


In [5]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [6]:
# Downloading LFW
import shutil

%cd "/content/drive/My Drive/stylegan2"
!wget http://vis-www.cs.umass.edu/lfw/lfw.tgz
shutil.unpack_archive("lfw.tgz", "/content/drive/My Drive/stylegan2")
# The shutil will take some of time to run

/content/drive/My Drive/stylegan2
--2021-02-04 10:28:27--  http://vis-www.cs.umass.edu/lfw/lfw.tgz
Resolving vis-www.cs.umass.edu (vis-www.cs.umass.edu)... 128.119.244.95
Connecting to vis-www.cs.umass.edu (vis-www.cs.umass.edu)|128.119.244.95|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 180566744 (172M) [application/x-gzip]
Saving to: ‘lfw.tgz’

lfw.tgz             100%[===================>] 172.20M  34.0MB/s    in 7.5s    

2021-02-04 10:28:35 (22.9 MB/s) - ‘lfw.tgz’ saved [180566744/180566744]



After the create_from_images argument you need to pass in two paths. The first path is where the .tfrecords files should be output (just edit the last part to have a unique name). The second path is to the directory of your images.

In [23]:
# Lets check for files if they exist and use popular list show command
%cd "/content/drive/My Drive/stylegan2"
!ls

/content/drive/My Drive/stylegan2
lfw  lfw.tgz  stylegan2


In [24]:
!mkdir lfw_combined

In [22]:
%cd lfw
!ls

/content/drive/My Drive/stylegan2/lfw
Aaron_Eckhart			 Joshua_Harapko
Aaron_Guiel			 Joshua_Perper
Aaron_Patterson			 Joxel_Garcia
Aaron_Peirsol			 Joy_Bryant
Aaron_Pena			 Joy_Lee_Sadler
Aaron_Sorkin			 JP_Suarez
Aaron_Tippin			 JT_Snow
Abba_Eban			 Juan_Antonio_Samaranch
Abbas_Kiarostami		 Juan_Carlos
Abdel_Aziz_Al-Hakim		 Juan_Carlos_Ferrero
Abdel_Madi_Shabneh		 Juan_Carlos_Morales
Abdel_Nasser_Assidi		 Juan_Carlos_Ortega
Abdoulaye_Wade			 Juanes
Abdulaziz_Kamilov		 Juan_Fernandez
Abdullah			 Juan_Francisco_Palencia
Abdullah_Ahmad_Badawi		 Juan_Ignacio_Chela
Abdullah_al-Attiyah		 Juan_Jose_Lucas
Abdullah_Gul			 Juan_Manuel_Marquez
Abdullah_Nasseef		 Juan_Pablo_Montoya
Abdullatif_Sener		 Juan_Roman_Carrasco
Abdul_Majeed_Shobokshi		 Juan_Roman_Riquelme
Abdul_Rahman			 Juan_Sanchez
Abel_Aguilar			 Juan_Valencia_Osorio
Abel_Pacheco			 Judd_Davies
Abid_Hamid_Mahmud_Al-Tikriti	 Jude_Law
Abner_Martinez			 Judi_Dench
Abraham_Foxman			 Judi_Patton
Aby_Har-Even			 Judith_Nathan
Adam_Ant			 Ju

In [29]:
!cp -r /content/drive/My\ Drive/stylegan2/lfw/*/*.jpg /content/drive/My\ Drive/stylegan2/lfw_combined

In [32]:
%cd "/content/drive/My Drive/stylegan2/stylegan2/"

/content/drive/My Drive/stylegan2/stylegan2


In [33]:
#2nd argument is where to put your tfrecords files
#3rd should point at your image dataset
!python dataset_tool.py create_from_images /content/drive/My\ Drive/stylegan2/stylegan2/datasets/lfwpkl /content/drive/My\ Drive/stylegan2/lfw_combined

Loading images from "/content/drive/My Drive/stylegan2/lfw_combined"
Error: Input image resolution must be a power-of-two


### Error: Input image resolution must be a power-of-two

### Lets convert all images to that particular size then...

In [34]:
%cd "/content/drive/My Drive/stylegan2/"

/content/drive/My Drive/stylegan2


In [36]:
# Creating backup for lfw_datasets
!cp -r lfw_combined lfw_combined_copy

In [37]:
!ls

lfw  lfw_combined  lfw_combined_copy  lfw.tgz  stylegan2


In [38]:
%cd "/content/drive/My Drive/stylegan2/lfw_combined"

/content/drive/My Drive/stylegan2/lfw_combined


In [46]:
#!/usr/bin/python
from PIL import Image
import os, sys

path = "/content/drive/My Drive/stylegan2/lfw_combined/"
dirs = os.listdir( path )

def resize():
    for item in dirs:
        if os.path.isfile(path+item):
            im = Image.open(path+item)
            f, e = os.path.splitext(path+item)
            imResize = im.resize((32,32), Image.ANTIALIAS)
            imResize.save(f + '.jpg', 'JPEG', quality=90)

resize()

In [47]:
!pwd

/content/drive/My Drive/stylegan2/stylegan2


In [49]:
%cd "/content/drive/My Drive/stylegan2/stylegan2/"
#2nd argument is where to put your tfrecords files
#3rd should point at your image dataset
!python dataset_tool.py create_from_images /content/drive/My\ Drive/stylegan2/stylegan2/datasets/lfwpkl/ /content/drive/My\ Drive/stylegan2/lfw_combined/

/content/drive/My Drive/stylegan2/stylegan2
Loading images from "/content/drive/My Drive/stylegan2/lfw_combined/"
100% 13233/13233 [00:25<00:00, 515.58it/s]


--dataset

This should be the name you used in the first path when converting your dataset.

--mirror-augment

Using this option loads some images at random mirrored horizontally (left-to-right). This might help if you have a very small dataset.

--metrics

METRICS DON’T MATTER. It’s art! Use your eyes. Set --metrics=None and live your life.

If you must use metrics, you have a few options. fid50k, the default, uses Frechet Inception Distance score. It’s what was used in StyleGAN1 and what most people know. It’s fine for images of animals and things, but it’s not great. ppl_wend is what StyleGAN2 prefers and claims to be more accurate. There are a bunch of other options but I’d recommend you stick with those. Note that both of these take 30–45minutes to run every time it runs so that cuts into your training time in Colab.

In [50]:
%cd "/content/drive/My Drive/stylegan2/stylegan2/datasets/lfwpkl/"
!ls

/content/drive/My Drive/stylegan2/stylegan2/datasets/lfwpkl
lfwpkl-r02.tfrecords  lfwpkl-r04.tfrecords  -r02.tfrecords  -r04.tfrecords
lfwpkl-r03.tfrecords  lfwpkl-r05.tfrecords  -r03.tfrecords  -r05.tfrecords


In [51]:
!pwd

/content/drive/My Drive/stylegan2/stylegan2/datasets/lfwpkl


In [52]:
!ls

lfwpkl-r02.tfrecords  lfwpkl-r04.tfrecords  -r02.tfrecords  -r04.tfrecords
lfwpkl-r03.tfrecords  lfwpkl-r05.tfrecords  -r03.tfrecords  -r05.tfrecords


changed the names from listed above names to 

lfwpkl-1

lfwpkl-2

lfwpkl-3

lfwpkl-4

In [58]:
%cd "/content/drive/My Drive/stylegan2/stylegan2/"
!ls

/content/drive/My Drive/stylegan2/stylegan2
datasets		results
dataset_tool.py		run_generator.py
dnnlib			run_metrics.py
Dockerfile		run_projector.py
docs			run_training.py
easy-install.sh		runway_model.py
LICENSE.txt		runway.yml
metrics			StyleGAN2_Google_Colab_Example.ipynb
pkl			StyleGAN2_projection_interpolation.ipynb
pretrained_networks.py	StyleGAN2_Projection.ipynb
projector.py		test_nvcc.cu
README.md		training
requirements.txt	USAGE.md


In [81]:
%cd "/content/drive/My Drive/stylegan2/stylegan2/datasets/lfwpkl/"
!ls

/content/drive/My Drive/stylegan2/stylegan2/datasets/lfwpkl
lfwpkl-1  lfwpkl-3  -r02.tfrecords  -r04.tfrecords
lfwpkl-2  lfwpkl-4  -r03.tfrecords  -r05.tfrecords


In [82]:
!mv lfwpkl-1 lfwpkl-1.tfrecords
!mv lfwpkl-2 lfwpkl-2.tfrecords
!mv lfwpkl-3 lfwpkl-3.tfrecords
!mv lfwpkl-4 lfwpkl-4.tfrecords

In [83]:
!ls

lfwpkl-1.tfrecords  lfwpkl-3.tfrecords	-r02.tfrecords	-r04.tfrecords
lfwpkl-2.tfrecords  lfwpkl-4.tfrecords	-r03.tfrecords	-r05.tfrecords


In [88]:
# Interrupted , this will take 180 days in standard version of colab. Cannot use colab pro outside US.
# Lets use the pretrained version of the FFHQ and use it for LFW Dataset
%cd "/content/drive/My Drive/stylegan2/stylegan2/"
!python run_training.py --num-gpus=1 --data-dir=./datasets --config=config-f --dataset=lfwpkl --mirror-augment=true --metrics=None

/content/drive/My Drive/stylegan2/stylegan2
Couldn't find valid snapshot, starting over
Local submit - run_dir: results/00014-stylegan2-lfwpkl-1gpu-config-f
dnnlib: Running training.training_loop.training_loop() on localhost...
Streaming data using training.dataset.TFRecordDataset...
tcmalloc: large alloc 4294967296 bytes == 0x5e78000 @  0x7fe137551001 0x7fe133ff74ff 0x7fe134047b08 0x7fe13404bac7 0x7fe1340ea1a3 0x50a4a5 0x50cc96 0x507be4 0x508ec2 0x594a01 0x549e8f 0x5515c1 0x59fd0e 0x50d256 0x507be4 0x588e5c 0x59fd0e 0x50d256 0x507be4 0x588e5c 0x59fd0e 0x50d256 0x5095c8 0x50a2fd 0x50beb4 0x5095c8 0x50a2fd 0x50beb4 0x507be4 0x588e5c 0x59fd0e
tcmalloc: large alloc 4294967296 bytes == 0x7fdf6daca000 @  0x7fe13754f1e7 0x7fe133ff741e 0x7fe134047c2b 0x7fe13404830f 0x7fe1340ea0a3 0x50a4a5 0x50cc96 0x507be4 0x509900 0x50a2fd 0x50cc96 0x507be4 0x509900 0x50a2fd 0x50cc96 0x507be4 0x509900 0x50a2fd 0x50cc96 0x5095c8 0x50a2fd 0x50beb4 0x507be4 0x509900 0x50a2fd 0x50beb4 0x507be4 0x509900 0x50a2fd 

In [90]:
!pip install opensimplex
!python run_generator.py generate-images --network=/content/drive/My\ Drive/stylegan2/stylegan2/results/pretrained/network-snapshot-10000.pkl --seeds=3875451-3876000 --truncation-psi=0.7

Local submit - run_dir: results/00015-generate-images
dnnlib: Running run_generator.generate_images() on localhost...
Loading networks from "/content/drive/My Drive/stylegan2/stylegan2/results/pretrained/network-snapshot-10000.pkl"...
Setting up TensorFlow plugin "fused_bias_act.cu": Preprocessing... Loading... Done.
Setting up TensorFlow plugin "upfirdn_2d.cu": Preprocessing... Loading... Done.
Generating image for seed 3875451 (1/550) ...
Generating image for seed 3875452 (2/550) ...
Generating image for seed 3875453 (3/550) ...
Generating image for seed 3875454 (4/550) ...
Generating image for seed 3875455 (5/550) ...
Generating image for seed 3875456 (6/550) ...
Generating image for seed 3875457 (7/550) ...
Generating image for seed 3875458 (8/550) ...
Generating image for seed 3875459 (9/550) ...
Generating image for seed 3875460 (10/550) ...
Generating image for seed 3875461 (11/550) ...
Generating image for seed 3875462 (12/550) ...
Generating image for seed 3875463 (13/550) ...

In [91]:
!pwd

/content/drive/My Drive/stylegan2/stylegan2


In [97]:
!zip -r generated-0.7.zip ./results/00015-generate-images

  adding: results/00015-generate-images/ (stored 0%)
  adding: results/00015-generate-images/submit_config.pkl (deflated 43%)
  adding: results/00015-generate-images/submit_config.txt (deflated 55%)
  adding: results/00015-generate-images/log.txt (deflated 88%)
  adding: results/00015-generate-images/seed3875451.png (deflated 0%)
  adding: results/00015-generate-images/seed3875452.png (deflated 0%)
  adding: results/00015-generate-images/seed3875453.png (deflated 0%)
  adding: results/00015-generate-images/seed3875454.png (deflated 0%)
  adding: results/00015-generate-images/seed3875455.png (deflated 0%)
  adding: results/00015-generate-images/seed3875456.png (deflated 0%)
  adding: results/00015-generate-images/seed3875457.png (deflated 0%)
  adding: results/00015-generate-images/seed3875458.png (deflated 0%)
  adding: results/00015-generate-images/seed3875459.png (deflated 0%)
  adding: results/00015-generate-images/seed3875460.png (deflated 0%)
  adding: results/00015-generate-image

In [98]:
!python run_generator.py generate-latent-walk --network=/content/drive/My\ Drive/stylegan2/stylegan2/results/pretrained/network-snapshot-10000.pkl --seeds=3,11,17,25,3 --frames 200 --truncation-psi=0.7

Local submit - run_dir: results/00016-generate-latent-walk
dnnlib: Running run_generator.generate_latent_walk() on localhost...
Loading networks from "/content/drive/My Drive/stylegan2/stylegan2/results/pretrained/network-snapshot-10000.pkl"...
Setting up TensorFlow plugin "fused_bias_act.cu": Preprocessing... Loading... Done.
Setting up TensorFlow plugin "upfirdn_2d.cu": Preprocessing... Loading... Done.
not enough values to generate walk
Generating image for step 0/204 ...
Generating image for step 1/204 ...
Generating image for step 2/204 ...
Generating image for step 3/204 ...
Generating image for step 4/204 ...
Generating image for step 5/204 ...
Generating image for step 6/204 ...
Generating image for step 7/204 ...
Generating image for step 8/204 ...
Generating image for step 9/204 ...
Generating image for step 10/204 ...
Generating image for step 11/204 ...
Generating image for step 12/204 ...
Generating image for step 13/204 ...
Generating image for step 14/204 ...
Generating 

In [108]:
#convert to video 
!ffmpeg -r 24 -i ./results/00016-generate-latent-walk/frame%05d.png -vcodec libx264 -pix_fmt yuv420p latent-walk-v2.mp4

ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lib

In [109]:
!pwd

/content/drive/My Drive/stylegan2/stylegan2
